In [8]:
import networkx as nx
import fileselect

edges = []

test_dataset="""4 5
1 2
3 1
3 2
4 3
4 2""".split("\n")

#with open(fileselect.gui_fname(), "r") as fp:
for line in test_dataset:
    splitline = line.strip().split(" ")
    edges.append(splitline)

g = nx.DiGraph()
for edge in edges:
    g.add_edge(edge[0],edge[1])
print(" ".join(nx.topological_sort(g)))

4 5 3 1 2


In [5]:
import networkx as nx
import fileselect

edges = []

with open(fileselect.gui_fname(), "r") as fp:
    for line in fp:
        splitline = line.strip().split(" -> ")
        origin = splitline[0].strip()
        dest = splitline[1].split(",")
        for d in dest:
            edges.append([origin,d])

g = nx.DiGraph()
for edge in edges:
    g.add_edge(edge[0],edge[1])
print(", ".join(nx.topological_sort(g)))

8, 6, 5, 14, 3, 7, 29, 28, 35, 24, 23, 20, 19, 17, 40, 15, 16, 26, 39, 33, 13, 12, 21, 34, 41, 48, 10, 1, 11, 36, 37, 43, 46, 18, 25, 27, 38, 0, 4, 22, 32, 44, 45, 47, 9, 31, 42


In [9]:
import numpy as np

In [11]:
np.zeros((3,2))

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [72]:
test_data ="""AACCTTGG
ACACTGTGA""".split("\n")

first_line = False
second_line = False

import fileselect

with open(fileselect.gui_fname(), "r") as fp:
    for line in fp:
        if second_line==False:
            second_line = line.strip()
        elif first_line==False:
            first_line = line.strip()
        
def LCSLength(X, Y):
    m = len(X)
    n = len(Y)
    C = np.zeros((m,n)) #array(0..m, 0..n)
    for i in range(m):
        C[i,0] = 0
    for j in range(n):
        C[0,j] = 0
    for i in range(0,m):
        for j in range(0,n):
            if X[i] == Y[j]:
                C[i,j] = C[i-1,j-1] + 1
            else:
                C[i,j] = max(C[i,j-1], C[i-1,j])
    return C,C[m-1,n-1]

def backtrack(C, X, Y, i, j):
    if i < 0 or j < 0:
        return ""
    if  X[i] == Y[j]:
        return backtrack(C, X, Y, i-1, j-1) + X[i]
    if C[i,j-1] > C[i-1,j]:
        return backtrack(C, X, Y, i, j-1)
    return backtrack(C, X, Y, i-1, j)

C,lengthlong = LCSLength(first_line,second_line)
backtrack(C,first_line,second_line,len(first_line)-1,len(second_line)-1)

'CTGCATCTTCACCCAACCAGGGGCTAGATAGCACAGTCTTCAACGTCAAGTACAAAGTATTCCGTCCGAAGCTATCGCCGCCTGTTAGCTTGAGGGGCGCCGGAAATTTCTATAGTTCCGACCCTTTGAGTGACATGCTAGGTTGATTAGGGTAAAACATAGTCGACGTTGTACTTTTAATCGGTCCGTTTCCTATGCCCCTTTTTGAAGTGGGTTTGATCGAGATACCTCTGAATCAAACTTGAAGTA'

In [31]:
import pandas as pd
import re
from Bio import pairwise2
from collections import defaultdict

score_mat ="""  A  C  D  E  F  G  H  I  K  L  M  N  P  Q  R  S  T  V  W  Y
A  4  0 -2 -1 -2  0 -2 -1 -1 -1 -1 -2 -1 -1 -1  1  0  0 -3 -2
C  0  9 -3 -4 -2 -3 -3 -1 -3 -1 -1 -3 -3 -3 -3 -1 -1 -1 -2 -2
D -2 -3  6  2 -3 -1 -1 -3 -1 -4 -3  1 -1  0 -2  0 -1 -3 -4 -3
E -1 -4  2  5 -3 -2  0 -3  1 -3 -2  0 -1  2  0  0 -1 -2 -3 -2
F -2 -2 -3 -3  6 -3 -1  0 -3  0  0 -3 -4 -3 -3 -2 -2 -1  1  3
G  0 -3 -1 -2 -3  6 -2 -4 -2 -4 -3  0 -2 -2 -2  0 -2 -3 -2 -3
H -2 -3 -1  0 -1 -2  8 -3 -1 -3 -2  1 -2  0  0 -1 -2 -3 -2  2
I -1 -1 -3 -3  0 -4 -3  4 -3  2  1 -3 -3 -3 -3 -2 -1  3 -3 -1
K -1 -3 -1  1 -3 -2 -1 -3  5 -2 -1  0 -1  1  2  0 -1 -2 -3 -2
L -1 -1 -4 -3  0 -4 -3  2 -2  4  2 -3 -3 -2 -2 -2 -1  1 -2 -1
M -1 -1 -3 -2  0 -3 -2  1 -1  2  5 -2 -2  0 -1 -1 -1  1 -1 -1
N -2 -3  1  0 -3  0  1 -3  0 -3 -2  6 -2  0  0  1  0 -3 -4 -2
P -1 -3 -1 -1 -4 -2 -2 -3 -1 -3 -2 -2  7 -1 -2 -1 -1 -2 -4 -3
Q -1 -3  0  2 -3 -2  0 -3  1 -2  0  0 -1  5  1  0 -1 -2 -2 -1
R -1 -3 -2  0 -3 -2  0 -3  2 -2 -1  0 -2  1  5 -1 -1 -3 -3 -2
S  1 -1  0  0 -2  0 -1 -2  0 -2 -1  1 -1  0 -1  4  1 -2 -3 -2
T  0 -1 -1 -1 -2 -2 -2 -1 -1 -1 -1  0 -1 -1 -1  1  5  0 -2 -2
V  0 -1 -3 -2 -1 -3 -3  3 -2  1  1 -3 -2 -2 -3 -2  0  4 -3 -1
W -3 -2 -4 -3  1 -2 -2 -3 -3 -2 -1 -4 -4 -2 -3 -3 -2 -3 11  2
Y -2 -2 -3 -2  3 -3  2 -1 -2 -1 -1 -2 -3 -1 -2 -2 -2 -1  2  7""".split("\n")
score_mat = [re.split(" +",line) for line in score_mat]
score_mat[0][0] = "index"

score_translate = defaultdict(dict)
score_pd = pd.DataFrame(score_mat[1:],columns=score_mat[0]).set_index("index")
for row in score_pd.index:
    for col in score_pd.columns:
        score_translate[row][col] = score_pd.loc[row,col]

In [32]:
score_translate

defaultdict(dict,
            {'A': {'A': '4',
              'C': '0',
              'D': '-2',
              'E': '-1',
              'F': '-2',
              'G': '0',
              'H': '-2',
              'I': '-1',
              'K': '-1',
              'L': '-1',
              'M': '-1',
              'N': '-2',
              'P': '-1',
              'Q': '-1',
              'R': '-1',
              'S': '1',
              'T': '0',
              'V': '0',
              'W': '-3',
              'Y': '-2'},
             'C': {'A': '0',
              'C': '9',
              'D': '-3',
              'E': '-4',
              'F': '-2',
              'G': '-3',
              'H': '-3',
              'I': '-1',
              'K': '-3',
              'L': '-1',
              'M': '-1',
              'N': '-3',
              'P': '-3',
              'Q': '-3',
              'R': '-3',
              'S': '-1',
              'T': '-1',
              'V': '-1',
              'W': '-2',
    

In [66]:
example_data = """>Rosalind_67
PLEASANTLY
>Rosalind_17
MEANLY""".split("\n")

first_line = ""
second_line = ""
linenum = 1
with open(fileselect.gui_fname(), "r") as fp:
    for line in fp:
        if linenum==2:
            first_line = line.strip()
        if linenum==4:
            second_line = line.strip()
        linenum += 1

In [76]:
import fileselect
from Bio.pairwise2 import format_alignment
from Bio.SubsMat import MatrixInfo as matlist
matrix = matlist.blosum62

whichline = None
first_line = ""
second_line = ""
linenum = 1
with open(fileselect.gui_fname(), "r") as fp:
    for line in fp:
        if line[0]==">": 
            if whichline==None:
                whichline="first"
            elif whichline=="first":
                whichline="second"
            continue
        if whichline=="first":
            first_line += line.strip()
        if whichline=="second":
            second_line += line.strip()

alignments = pairwise2.align.globalds(first_line,second_line,matrix,-5,-5)
scores = [int(i[2]) for i in alignments]
print(max(scores))




1869
